In [25]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
np.set_printoptions(suppress=True)
movies = pd.read_csv('movies.csv', low_memory=False)
ratings = pd.read_csv('ratings.csv', low_memory=False)
asc = ratings.sort_values(by=['userId', 'timestamp'])

def getListUid(uid):
    ratings_temp = ratings    
    final_dataset = ratings_temp.pivot(index='movieId',columns='userId',values='rating')
    final_dataset = final_dataset.fillna(final_dataset.mean(axis=0))
    desc = ratings_temp[ratings_temp['userId'] == uid].sort_values(by='timestamp', ascending=False)
    arr =desc.head(5).values
    movie_list = arr[:,1]
    print(movie_list)
    final_dataset.fillna(0,inplace=True)
    no_user_voted = ratings_temp.groupby('movieId')['rating'].agg('count')
    no_movies_voted = ratings_temp.groupby('userId')['rating'].agg('count')
    print(no_user_voted)
    print(no_movies_voted)
    final_dataset = final_dataset.loc[no_user_voted[no_user_voted>3].index,:]
    final_dataset = final_dataset.loc[:,no_movies_voted[no_movies_voted > 50].index]
    final_dataset = final_dataset.loc[movie_list]

    final_dataset = final_dataset.T
    new_arr=  np.array(final_dataset)
    list_data = new_arr.tolist()
    knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)
    knn.fit(list_data)
    distances , indices =knn.kneighbors([list_data[0]],n_neighbors=10)
    return sorted(zip(distances.tolist()[0], indices.tolist()[0]))

In [26]:
def get_recommendation(uid):
    list_uid = getListUid(uid)
    print(list_uid)
    list_uid.pop(0)
    ratings_temp = ratings
    test_id = ratings_temp[ratings_temp['userId'] == uid]

    x = int(test_id.size/4)
    count_phim = []
    for i in range(x):
        if (test_id.iloc[i].at['rating'] >= 4):
            count_phim.append(test_id.iloc[i].at['movieId'])
    count_phim = []
    np.set_printoptions(suppress=True,\
    formatter={'float_kind':'{:0.0f}'.format})
    for i in list_uid:
        test_id = ratings_temp[ratings_temp['userId'] == i[1]]
        x = int(test_id.size/4)
        for j in range(x):
            if (test_id.iloc[j].at['rating'] >= 4):
                count_phim.append(test_id.iloc[j].at['movieId'])
    unique, counts = np.unique(np.array(count_phim), return_counts=True)
    final_arr = np.asarray((unique, counts)).T.tolist()
    top_recommendation = sorted(final_arr,key=lambda l:l[1], reverse=True)
    list_phim = []
    for i in range(10):
        list_phim.append(top_recommendation[i][0])
    list_movie_name = []
    for i in list_phim:
        list_movie_name.append(movies.loc[i]['title'])
    return list_movie_name
    

print(get_recommendation(19))

[   527. 136018.   2324.   1584.  30707.]
movieId
1         232
2          92
3          58
4          11
5          62
         ... 
146684      1
146878      1
148238      1
148626      3
149532      1
Name: rating, Length: 10325, dtype: int64
userId
1       113
2        29
3        73
4       124
5        68
       ... 
664      74
665     227
666     225
667      74
668    5678
Name: rating, Length: 668, dtype: int64


KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Float64Index([136018.0], dtype='float64', name='movieId'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"